In [1]:
print ("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\shalo\\Documents\\Healio\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shalo\\Documents\\Healio'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 18939


In [13]:
# text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
# query_result

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "healio"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [24]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [25]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [26]:
# Load Existing index 

from langchain_pinecone import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs = retriever.invoke("What are Calluses?")

In [30]:
retrieved_docs

[Document(metadata={'page': 550.0, 'source': 'data\\Medical_book.pdf'}, page_content='cally to boils to fight infection. These include essential\noils of bergamot (Citrus bergamia ), chamomile ( Matri-\nGALE ENCYCLOPEDIA OF MEDICINE 2 537\nBoils\nA close-up view of a carbuncle on person’s back.(Photo-\ngraph by John Watney, Photo Researchers, Inc. Reproduced by\npermission.)\nGEM -0433 to 0624 - B  10/22/03 6:08 PM  Page 537'),
 Document(metadata={'page': 390.0, 'source': 'data\\Medical_book.pdf'}, page_content='immune system is suppressed or not functioning\nproperly.\nMeningitis—Inflammation of the membranes cov-\nering the brain and spinal cord, called the\nmeninges.\nNebulizer—A device that produces an extremely\nfine mist that is readily inhalable.\nSpores—The small, thick-walled reproductive\nstructures of fungi.\nSputum—Mucus and other matter coughed up\nfrom the airways.\nORGANIZATIONS\nAmerican College of Allergy, Asthma, and Immunology. 85\nWest Algonquin Road, Suite 550, Arl

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.4, max_tokens=500)

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input": "What is Shingles?"})
print(response["answer"])

Shingles is a disease caused by the herpes zoster virus, the same virus that causes chickenpox.  It's characterized by pain and blisters along one nerve, usually on the face, chest, stomach, or back.  The name "shingles" comes from the Latin word "cingulum," meaning girdle, reflecting its common thoracic presentation.


In [43]:
response = rag_chain.invoke({"input": "What is mathematics?"})
print(response["answer"])

This document does not contain the answer to what mathematics is.  It discusses neuropsychological tests, art therapy, and arthroscopic surgery.  It does not define mathematics.
